In [1]:
!git clone https://github.com/AmirHosseinNaghdi2003/DGM-Project

Cloning into 'DGM-Project'...
remote: Enumerating objects: 93, done.
remote: Total 93 (delta 0), reused 0 (delta 0), pack-reused 93 (from 1)
Receiving objects: 100% (93/93), 42.13 MiB | 20.18 MiB/s, done.
Resolving deltas: 100% (36/36), done.


DGM

In [6]:
import os
import yaml
import torch
import pickle
import numpy as np
from models.mart import MART
from types import SimpleNamespace
from torch.utils.data import Dataset

In [8]:
!nvidia-smi

Thu Jan 16 13:56:52 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8               9W /  70W |      3MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
# Install the correct version of PyTorch with GPU support
!pip install torch==2.1.0+cu118 torchvision==0.15.0+cu118 torchaudio==2.1.0+cu118 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 GB 420.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 54.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 62.8 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
ERROR: Cannot install torch==2.1.0+cu118 and torchvision==0.15.0+cu118 because these package versions have conflicting dependencies.

The conflict is caused by:
    The user requested torch==2.1.0+cu118
    torchvision 0.15.0+cu118 depends on torch==2.0.0+cu118

To fix this you could try to:
1. loosen the range of package versions you've specified
2. remove package versions to allow pip to attempt to solve the dependency conflict

ERROR: ResolutionImpossible: for help visit https://pip.pypa.io/en/latest/topics/dependency-resolution/#dealin

In [ ]:
import torch
print(torch.cuda.is_available())

True


In [9]:
def load_config(config_path):
    """
    Load configuration from a YAML file.

    Args:
        config_path (str): Path to the YAML configuration file.

    Returns:
        dict: Loaded configuration dictionary.
    """
    print(f"[INFO] Loading configuration from {config_path}...")
    with open(config_path, 'r') as file:
        config = yaml.safe_load(file)
    print("[INFO] Configuration loaded successfully!")
    return config


In [10]:
CHECKPOINT_PATH = "files/sdd_ckpt_best.pth"  # Update this
TEST_DATA_PATH = "files/sdd_test.pkl"  # Update this
CONFIG_PATH = "files/mart_sdd_reproduce.yaml"  # Update this
BATCH_SIZE = 64

In [11]:
def load_model(checkpoint_path, opts):
    print("[INFO] Loading model from checkpoint...")
    args = SimpleNamespace(**opts)
    model = MART(args)  # Remove .cuda()
    checkpoint = torch.load(checkpoint_path, map_location='cpu', weights_only=True)


    model.load_state_dict(checkpoint['state_dict'])  # or whatever loading method you are using

    print("[INFO] Model loaded successfully!")
    return model


In [12]:
def evaluate_model(model, test_loader, opts):
    print("[INFO] Starting evaluation...")
    ade_sum, fde_sum, total_agents = 0.0, 0.0, 0

    # Set the device to GPU or CPU
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)  # Move model to device

    with torch.no_grad():
        for sample in test_loader:
            x_abs, y = sample
            x_abs, y = x_abs.to(device), y.to(device)  # Move tensors to device

            batch_size, num_agents, length, _ = x_abs.size()

            x_rel = torch.zeros_like(x_abs).to(device)  # Ensure x_rel is on the same device
            x_rel[:, :, 1:] = x_abs[:, :, 1:] - x_abs[:, :, :-1]
            x_rel[:, :, 0] = x_rel[:, :, 1]

            y_pred = model(x_abs, x_rel)

            if opts["pred_rel"]:
                cur_pos = x_abs[:, :, [-1]].unsqueeze(2)
                y_pred = torch.cumsum(y_pred, dim=3) + cur_pos

            ade = torch.min(torch.mean(torch.norm(y_pred - y[:, :, None], dim=-1), dim=3), dim=2)[0].mean().item()
            fde = torch.min(torch.mean(torch.norm(y_pred[:, :, :, -1:] - y[:, :, None, -1:], dim=-1), dim=3), dim=2)[0].mean().item()

            ade_sum += ade * num_agents * batch_size
            fde_sum += fde * num_agents * batch_size
            total_agents += num_agents * batch_size

    ade_avg = (ade_sum / total_agents) * opts["scale"]
    fde_avg = (fde_sum / total_agents) * opts["scale"]

    print(f"[INFO] Evaluation Results: ADE = {ade_avg:.4f}, FDE = {fde_avg:.4f}")
    return ade_avg, fde_avg


In [13]:
import torch
import numpy as np

class TrajectoryDataset(Dataset):
    def __init__(
        self, obs_len=8, pred_len=12, mode='train', scale=10, inputs=None, max_agents=50
    ):
        super(TrajectoryDataset, self).__init__()

        self.obs_len = obs_len
        self.pred_len = pred_len
        self.seq_len = self.obs_len + self.pred_len
        self.scale = scale
        self.max_agents = max_agents

        with open('files/sdd_test.pkl'.format(mode), 'rb') as f:
            traj = pickle.load(f)

        traj_tmp = []

        for t in traj:
            traj_tmp.append(t)
            if mode == 'train':
                traj_tmp.append(np.flip(t, axis=1))

        self.traj = []
        if 'pos_x' in inputs and 'pos_y' in inputs:
            for t in traj_tmp:
                t -= t[:, :1, :]
                self.traj.append(t)
        else:
            self.traj = traj_tmp


    def __len__(self):
        return len(self.traj)

    def __getitem__(self, index):
        past_traj = self.traj[index][:, :self.obs_len] * self.scale
        future_traj = self.traj[index][:, self.obs_len:] * self.scale

        # Padding the number of agents to max_agents if necessary
        num_agents = past_traj.shape[0]
        if num_agents < self.max_agents:
            pad_size = self.max_agents - num_agents
            past_traj_padded = np.pad(past_traj, ((0, pad_size), (0, 0), (0, 0)), mode='constant')
            future_traj_padded = np.pad(future_traj, ((0, pad_size), (0, 0), (0, 0)), mode='constant')
        else:
            past_traj_padded = past_traj
            future_traj_padded = future_traj

        past_traj = torch.from_numpy(past_traj_padded).type(torch.float)
        future_traj = torch.from_numpy(future_traj_padded).type(torch.float)

        return [past_traj, future_traj]


In [14]:
from torch.utils.data import DataLoader
if __name__ == "__main__":
    opts = load_config(CONFIG_PATH)
    model = load_model(CHECKPOINT_PATH, opts)

    # Prepare the test dataset and DataLoader
    test_dataset = TrajectoryDataset(
        obs_len=opts["past_length"],
        pred_len=opts["future_length"],
        mode="test",  # This loads the test set
        scale=opts["scale"],
        inputs=opts["inputs"]
    )
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

    print("[INFO] Starting evaluation with test data...")
    evaluate_model(model, test_loader, opts)

[INFO] Loading configuration from files/mart_sdd_reproduce.yaml...
[INFO] Configuration loaded successfully!
[INFO] Loading model from checkpoint...
[INFO] PRT Agg: cat
[INFO] HRT Agg: avg
[INFO] Binary Threshold Function Type: 2
[INFO] Model loaded successfully!
[INFO] Starting evaluation with test data...
[INFO] Starting evaluation...


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


[INFO] Evaluation Results: ADE = 0.7536, FDE = 1.1946


In [15]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from copy import deepcopy
from collections import OrderedDict
import yaml
from types import SimpleNamespace
import importlib
import warnings
import os

In [19]:
# Import the updated TrajectoryDataset and seq_collate
import dataloader_eth  # Replace with the correct module name if different
importlib.reload(dataloader_eth)
from dataloader_eth import TrajectoryDataset, seq_collate

In [20]:
# Import the custom loss function
class MinMSELoss(nn.Module):
    def __init__(self, reduction='mean'):
        super(MinMSELoss, self).__init__()
        self.reduction = reduction
        self.mse = nn.MSELoss(reduction='none')

    def forward(self, inputs, targets):
        """
        Args:
            inputs: Tensor of shape [batch_size, max_agents, num_preds, pred_len, 2]
            targets: Tensor of shape [batch_size, max_agents, pred_len, 2]
        Returns:
            Scalar loss
        """
        # Compute MSE loss without reduction
        mse = self.mse(inputs, targets.unsqueeze(2))  # Shape: [batch_size, max_agents, num_preds, pred_len, 2]
        mse = mse.mean(dim=-1)  # Shape: [batch_size, max_agents, num_preds, pred_len]
        mse = mse.mean(dim=-1)  # Shape: [batch_size, max_agents, num_preds]

        # Select the minimum loss across predictions
        min_mse, _ = mse.min(dim=-1)  # Shape: [batch_size, max_agents]

        if self.reduction == 'mean':
            return min_mse.mean()
        elif self.reduction == 'sum':
            return min_mse.sum()
        else:
            return min_mse

In [21]:
# Load model configuration
with open('files/mart_sdd_reproduce.yaml', 'r') as file:
    config_dict = yaml.safe_load(file)

In [22]:
# Convert the config dictionary to an object with attributes
config = SimpleNamespace(**config_dict)

In [23]:
# Ensure the base model is loaded from the SDD checkpoint
from models.mart import MART  # Ensure this import is correct

def load_model(checkpoint_path, opts):
    print("[INFO] Loading model from checkpoint...")
    args = opts  # Directly use the SimpleNamespace without unpacking
    model = MART(args)
    # Handle FutureWarning by setting weights_only=True if possible
    try:
        checkpoint = torch.load(checkpoint_path, map_location='cpu', weights_only=True)
    except TypeError:
        # If weights_only is not supported, suppress the warning
        warnings.filterwarnings("ignore", category=FutureWarning)
        checkpoint = torch.load(checkpoint_path, map_location='cpu')
    # Ensure the correct key is used based on how the model was saved
    if 'model_state_dict' in checkpoint:
        model.load_state_dict(checkpoint['model_state_dict'])
    elif 'state_dict' in checkpoint:
        model.load_state_dict(checkpoint['state_dict'])
    else:
        raise KeyError("Checkpoint does not contain 'model_state_dict' or 'state_dict'.")
    print("[INFO] Model loaded successfully!")
    return model

In [24]:
CHECKPOINT_PATH = "files/sdd_ckpt_best.pth"  # Ensure this path is correct
base_model = load_model(CHECKPOINT_PATH, config)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
base_model.to(device)

[INFO] Loading model from checkpoint...
[INFO] PRT Agg: cat
[INFO] HRT Agg: avg
[INFO] Binary Threshold Function Type: 2
[INFO] Model loaded successfully!


MART(
  (input_fc): Linear(in_features=2, out_features=64, bias=True)
  (input_fc2): Linear(in_features=512, out_features=64, bias=True)
  (pos_encoder): PositionalAgentEncoding(
    (dropout): Dropout(p=0.1, inplace=False)
    (fc): Linear(in_features=128, out_features=64, bias=True)
  )
  (pair_encoders): ModuleList(
    (0): RT(
      (layers): ModuleList(
        (0): RTTransformerLayer(
          (attention_layer): RTAttentionLayer(
            (proj_qkv_n): Linear(in_features=64, out_features=192, bias=True)
            (proj_qkv_e): Linear(in_features=64, out_features=192, bias=True)
            (proj_o): Linear(in_features=64, out_features=64, bias=True)
          )
          (dropout): Dropout(p=0.0, inplace=False)
          (linear_net_n): Sequential(
            (0): Linear(in_features=64, out_features=128, bias=True)
            (1): ReLU(inplace=True)
            (2): Linear(in_features=128, out_features=64, bias=True)
          )
          (norm1_n): LayerNorm((64,), eps=

In [25]:
# Define datasets for ETH-UCY (organized as directories)
datasets = {
    'eth': 'Data/eth',
    'hotel': 'Data/hotel',
    'univ': 'Data/univ',  # Corrected path
    'zara1': 'Data/zara1',
    'zara2': 'Data/zara2',
}

In [26]:
# Prepare data loaders for each subset
data_loaders = {}
for subset, path in datasets.items():
    # Check if the path exists
    if not os.path.isdir(path):
        print(f"[ERROR] Directory not found: {path}. Please ensure the dataset is correctly downloaded and extracted.")
        continue  # Skip this subset

    dataset = TrajectoryDataset(
        args=config,
        data_dir=path,
        obs_len=config.past_length,
        pred_len=config.future_length,
        delim='\t',
        max_agents=60,  # Increased to cover maximum agents
    )
    loader = DataLoader(
        dataset,
        batch_size=config.batch_size,
        shuffle=True,
        collate_fn=seq_collate,
        num_workers=2,  # Reduced to 2 based on system recommendation
    )
    data_loaders[subset] = loader

Processing Data .....


100%|██████████| 253/253 [00:00<00:00, 1647.31it/s]


Processing Data .....


100%|██████████| 445/445 [00:00<00:00, 1174.36it/s]


[ERROR] Directory not found: Data/univ. Please ensure the dataset is correctly downloaded and extracted.
Processing Data .....


100%|██████████| 705/705 [00:00<00:00, 1059.87it/s]


Processing Data .....


100%|██████████| 695/695 [00:00<00:00, 984.08it/s]


In [27]:
# Verify that at least one DataLoader is available
if not data_loaders:
    raise ValueError("No valid dataset directories found. Please check your dataset paths.")

def fine_tune_model(model, data_loader, optimizer, criterion, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        epoch_loss = 0.0
        for batch_idx, batch in enumerate(data_loader):
            # Ensure that 'past_traj' and 'future_traj' are keys returned by seq_collate
            inputs = batch['past_traj'].to(device)      # Shape: [batch_size, max_agents, obs_len, 2]
            targets = batch['future_traj'].to(device)   # Shape: [batch_size, max_agents, pred_len, 2]

            # Debugging statements
            if batch_idx == 0:
                print(f"Epoch {epoch+1}, Batch {batch_idx+1}:")
                print(f"  Inputs shape: {inputs.shape}")
                print(f"  Targets shape: {targets.shape}")

            # Compute relative positions if required by the model
            x_rel = torch.zeros_like(inputs).to(device)
            x_rel[:, :, 1:] = inputs[:, :, 1:] - inputs[:, :, :-1]
            x_rel[:, :, 0] = x_rel[:, :, 1]

            # Forward pass
            outputs = model(inputs, x_rel)  # Adjust if your model requires different inputs

            # Compute loss
            loss = criterion(outputs, targets)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()
        avg_loss = epoch_loss / len(data_loader)
        print(f"Epoch [{epoch+1}/{num_epochs}] Loss: {avg_loss:.4f}")
    return deepcopy(model.state_dict())


In [28]:
def average_weights(weight_list):
    avg_weights = OrderedDict()
    for key in weight_list[0].keys():
        # Stack weights for the current key from all models and compute the mean
        avg_weights[key] = torch.mean(torch.stack([weights[key] for weights in weight_list]), dim=0)
    return avg_weights

fine_tuned_weights = []

In [ ]:
import os
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Directory to save checkpoints in Google Drive
checkpoint_dir = "/content/drive/My Drive/FineTunedCheckpoints"
os.makedirs(checkpoint_dir, exist_ok=True)

# Fine-tune on each subset separately
for subset_name, data_loader in data_loaders.items():
    print(f"\nFine-tuning on subset: {subset_name}")

    # Create a fresh copy of the base model for each subset
    model_copy = deepcopy(base_model)
    model_copy.to(device)

    # Define a separate optimizer for each model copy
    optimizer = torch.optim.Adam(model_copy.parameters(), lr=config.lr)

    # Define the loss criterion
    criterion = MinMSELoss(reduction='mean')  # Replace nn.MSELoss with your custom loss function

    # Fine-tune the model copy on the current subset
    state_dict = fine_tune_model(model_copy, data_loader, optimizer, criterion, num_epochs=config.num_epochs)

    # Save the checkpoint
    checkpoint_path = os.path.join(checkpoint_dir, f"{subset_name}_checkpoint.pth")
    torch.save(state_dict, checkpoint_path)
    print(f"Checkpoint saved for subset {subset_name} at {checkpoint_path}")

    # Collect the fine-tuned weights
    fine_tuned_weights.append(state_dict)
    print(f"Finished fine-tuning on subset: {subset_name}")

if not fine_tuned_weights:
    raise ValueError("No models were fine-tuned. Please check your data loaders and dataset directories.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

Fine-tuning on subset: eth
Epoch 1, Batch 1:
  Inputs shape: torch.Size([64, 60, 8, 2])
  Targets shape: torch.Size([64, 60, 12, 2])


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([64, 60, 1, 12, 2])) that is different to the input size (torch.Size([64, 60, 20, 12, 2])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([61, 60, 1, 12, 2])) that is different to the input size (torch.Size([61, 60, 20, 12, 2])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [1/300] Loss: 0.3717
Epoch 2, Batch 1:
  Inputs shape: torch.Size([64, 60, 8, 2])
  Targets shape: torch.Size([64, 60, 12, 2])
Epoch [2/300] Loss: 0.1946
Epoch 3, Batch 1:
  Inputs shape: torch.Size([64, 60, 8, 2])
  Targets shape: torch.Size([64, 60, 12, 2])
Epoch [3/300] Loss: 0.1499
Epoch 4, Batch 1:
  Inputs shape: torch.Size([64, 60, 8, 2])
  Targets shape: torch.Size([64, 60, 12, 2])
Epoch [4/300] Loss: 0.1155
Epoch 5, Batch 1:
  Inputs shape: torch.Size([64, 60, 8, 2])
  Targets shape: torch.Size([64, 60, 12, 2])
Epoch [5/300] Loss: 0.0985
Epoch 6, Batch 1:
  Inputs shape: torch.Size([64, 60, 8, 2])
  Targets shape: torch.Size([64, 60, 12, 2])
Epoch [6/300] Loss: 0.0916
Epoch 7, Batch 1:
  Inputs shape: torch.Size([64, 60, 8, 2])
  Targets shape: torch.Size([64, 60, 12, 2])
Epoch [7/300] Loss: 0.0835
Epoch 8, Batch 1:
  Inputs shape: torch.Size([64, 60, 8, 2])
  Targets shape: torch.Size([64, 60, 12, 2])
Epoch [8/300] Loss: 0.0709
Epoch 9, Batch 1:
  Inputs shape: torch.Si

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([1, 60, 1, 12, 2])) that is different to the input size (torch.Size([1, 60, 20, 12, 2])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 2, Batch 1:
  Inputs shape: torch.Size([64, 60, 8, 2])
  Targets shape: torch.Size([64, 60, 12, 2])
Epoch [2/300] Loss: 0.1466
Epoch 3, Batch 1:
  Inputs shape: torch.Size([64, 60, 8, 2])
  Targets shape: torch.Size([64, 60, 12, 2])
Epoch [3/300] Loss: 0.1027
Epoch 4, Batch 1:
  Inputs shape: torch.Size([64, 60, 8, 2])
  Targets shape: torch.Size([64, 60, 12, 2])
Epoch [4/300] Loss: 0.0800
Epoch 5, Batch 1:
  Inputs shape: torch.Size([64, 60, 8, 2])
  Targets shape: torch.Size([64, 60, 12, 2])
Epoch [5/300] Loss: 0.0682
Epoch 6, Batch 1:
  Inputs shape: torch.Size([64, 60, 8, 2])
  Targets shape: torch.Size([64, 60, 12, 2])
Epoch [6/300] Loss: 0.0614
Epoch 7, Batch 1:
  Inputs shape: torch.Size([64, 60, 8, 2])
  Targets shape: torch.Size([64, 60, 12, 2])
Epoch [7/300] Loss: 0.0583
Epoch 8, Batch 1:
  Inputs shape: torch.Size([64, 60, 8, 2])
  Targets shape: torch.Size([64, 60, 12, 2])
Epoch [8/300] Loss: 0.0626
Epoch 9, Batch 1:
  Inputs shape: torch.Size([64, 60, 8, 2])
  Target

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([55, 60, 1, 12, 2])) that is different to the input size (torch.Size([55, 60, 20, 12, 2])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [1/300] Loss: 0.5708
Epoch 2, Batch 1:
  Inputs shape: torch.Size([64, 60, 8, 2])
  Targets shape: torch.Size([64, 60, 12, 2])
Epoch [2/300] Loss: 0.1647
Epoch 3, Batch 1:
  Inputs shape: torch.Size([64, 60, 8, 2])
  Targets shape: torch.Size([64, 60, 12, 2])
Epoch [3/300] Loss: 0.1084
Epoch 4, Batch 1:
  Inputs shape: torch.Size([64, 60, 8, 2])
  Targets shape: torch.Size([64, 60, 12, 2])
Epoch [4/300] Loss: 0.0846
Epoch 5, Batch 1:
  Inputs shape: torch.Size([64, 60, 8, 2])
  Targets shape: torch.Size([64, 60, 12, 2])
Epoch [5/300] Loss: 0.0759
Epoch 6, Batch 1:
  Inputs shape: torch.Size([64, 60, 8, 2])
  Targets shape: torch.Size([64, 60, 12, 2])
Epoch [6/300] Loss: 0.0703
Epoch 7, Batch 1:
  Inputs shape: torch.Size([64, 60, 8, 2])
  Targets shape: torch.Size([64, 60, 12, 2])
Epoch [7/300] Loss: 0.0677
Epoch 8, Batch 1:
  Inputs shape: torch.Size([64, 60, 8, 2])
  Targets shape: torch.Size([64, 60, 12, 2])
Epoch [8/300] Loss: 0.0653
Epoch 9, Batch 1:
  Inputs shape: torch.Si

In [ ]:
# Perform weight averaging across all fine-tuned models
print("\nAveraging weights from all fine-tuned models...")
final_weights = average_weights(fine_tuned_weights)

# Load the averaged weights into the base model
base_model.load_state_dict(final_weights)

# Save the fine-tuned and averaged model
torch.save({'model_state_dict': base_model.state_dict()}, 'files/eth_ucy_finetuned.pth')
print("[INFO] Fine-tuned model saved successfully at 'files/eth_ucy_finetuned.pth'.")

# Save directory in Google Drive
averaged_model_dir = "/content/drive/My Drive/FineTunedCheckpoints"
os.makedirs(averaged_model_dir, exist_ok=True)

# Save the fine-tuned and averaged model to Google Drive
averaged_model_path = os.path.join(averaged_model_dir, "averaged_model_checkpoint.pth")
torch.save({'model_state_dict': base_model.state_dict()}, averaged_model_path)
print(f"[INFO] Fine-tuned averaged model saved successfully at: {averaged_model_path}")


Averaging weights from all fine-tuned models...
[INFO] Fine-tuned model saved successfully at 'files/eth_ucy_finetuned.pth'.
[INFO] Fine-tuned averaged model saved successfully at: /content/drive/My Drive/FineTunedCheckpoints/averaged_model_checkpoint.pth


In [ ]:
def evaluate_finetuned_model(model, test_loader, opts):
    print("[INFO] Starting evaluation of the fine-tuned model...")
    ade_sum, fde_sum, total_agents = 0.0, 0.0, 0

    model.eval()
    with torch.no_grad():
        for sample in test_loader:
            inputs = sample['past_traj'].to(device)    # Shape: [batch_size, max_agents, obs_len, 2]
            targets = sample['future_traj'].to(device)   # Shape: [batch_size, max_agents, pred_len, 2]

            batch_size, num_agents, length, _ = inputs.size()

            x_rel = torch.zeros_like(inputs).to(device)
            x_rel[:, :, 1:] = inputs[:, :, 1:] - inputs[:, :, :-1]
            x_rel[:, :, 0] = x_rel[:, :, 1]

            y_pred = model(inputs, x_rel)

            if opts.pred_rel:
                cur_pos = inputs[:, :, [-1]].unsqueeze(2)
                y_pred = torch.cumsum(y_pred, dim=3) + cur_pos

            ade = torch.min(torch.mean(torch.norm(y_pred - targets[:, :, None], dim=-1), dim=3), dim=2)[0].mean().item()
            fde = torch.min(torch.mean(torch.norm(y_pred[:, :, :, -1:] - targets[:, :, None, -1:], dim=-1), dim=3), dim=2)[0].mean().item()

            ade_sum += ade * num_agents * batch_size
            fde_sum += fde * num_agents * batch_size
            total_agents += num_agents * batch_size

    ade_avg = (ade_sum / total_agents) * opts.scale
    fde_avg = (fde_sum / total_agents) * opts.scale

    print(f"[INFO] Fine-Tuned Model Evaluation Results: ADE = {ade_avg:.4f}, FDE = {fde_avg:.4f}")
    return ade_avg, fde_avg

# Prepare the test DataLoader
test_dataset = TrajectoryDataset(
    args=config,
    data_dir='Data/eth_ucy_test',  # Ensure this is a directory containing test `.txt` files
    obs_len=config.past_length,
    pred_len=config.future_length,
    delim='\t',
    max_agents=50,  # Ensure consistency with training
)
test_loader = DataLoader(
    test_dataset,
    batch_size=config.batch_size,
    shuffle=False,
    collate_fn=seq_collate,
    num_workers=2,  # Reduced to 2
)

# Evaluate the fine-tuned model
evaluate_finetuned_model(base_model, test_loader, config)

[WARNING] max_agents (50) is less than the maximum number of agents in any frame (51). Consider increasing max_agents.
Processing Data .....


100%|██████████| 253/253 [00:00<00:00, 1422.95it/s]

[INFO] Starting evaluation of the fine-tuned model...


[INFO] Fine-Tuned Model Evaluation Results: ADE = 0.4566, FDE = 0.8376


(0.45659330135748794, 0.8375541187322187)

In [13]:
import os
import yaml
import torch
import pickle
import numpy as np
from models.mart import MART
from types import SimpleNamespace
from torch.utils.data import Dataset, DataLoader
from copy import deepcopy
from collections import OrderedDict
import importlib
import warnings
import torch.nn as nn

# ============================
# Initial Setup
# ============================

# Check GPU availability
!nvidia-smi

# Install the correct version of PyTorch with GPU support
!pip install torch==2.1.0+cu118 torchvision==0.15.0+cu118 torchaudio==2.1.0+cu118 -f https://download.pytorch.org/whl/torch_stable.html

import torch
print(torch.cuda.is_available())

def load_config(config_path):
    """
    Load configuration from a YAML file.

    Args:
        config_path (str): Path to the YAML configuration file.

    Returns:
        SimpleNamespace: Loaded configuration as a SimpleNamespace object.
    """
    print(f"[INFO] Loading configuration from {config_path}...")
    with open(config_path, 'r') as file:
        config = yaml.safe_load(file)
    print("[INFO] Configuration loaded successfully!")
    return SimpleNamespace(**config)

# Paths and Constants
CHECKPOINT_PATH = "files/sdd_ckpt_best.pth"          # Base checkpoint
TEST_DATA_PATH = "files/sdd_test.pkl"                # Test data
CONFIG_PATH = "files/mart_sdd_reproduce.yaml"        # Configuration file
BATCH_SIZE = 64

# ============================
# Dataset Definition
# ============================

class TrajectoryDataset(Dataset):
    def __init__(
        self, obs_len=8, pred_len=12, mode='train', scale=10, inputs=None,
        max_agents=50, data_dir=None, delim='\t'
    ):
        super(TrajectoryDataset, self).__init__()

        self.obs_len = obs_len
        self.pred_len = pred_len
        self.seq_len = self.obs_len + self.pred_len
        self.scale = scale
        self.max_agents = max_agents

        if data_dir is None:
            data_dir = 'files/sdd_test.pkl'
        with open(data_dir, 'rb') as f:
            traj = pickle.load(f)

        traj_tmp = []

        for t in traj:
            traj_tmp.append(t)
            if mode == 'train':
                traj_tmp.append(np.flip(t, axis=1))

        self.traj = []
        if inputs and 'pos_x' in inputs and 'pos_y' in inputs:
            for t in traj_tmp:
                t -= t[:, :1, :]
                self.traj.append(t)
        else:
            self.traj = traj_tmp

    def __len__(self):
        return len(self.traj)

    def __getitem__(self, index):
        past_traj = self.traj[index][:, :self.obs_len] * self.scale
        future_traj = self.traj[index][:, self.obs_len:] * self.scale

        # Padding the number of agents to max_agents if necessary
        num_agents = past_traj.shape[0]
        if num_agents < self.max_agents:
            pad_size = self.max_agents - num_agents
            past_traj_padded = np.pad(
                past_traj, ((0, pad_size), (0, 0), (0, 0)), mode='constant'
            )
            future_traj_padded = np.pad(
                future_traj, ((0, pad_size), (0, 0), (0, 0)), mode='constant'
            )
        else:
            past_traj_padded = past_traj
            future_traj_padded = future_traj

        past_traj = torch.from_numpy(past_traj_padded).type(torch.float)
        future_traj = torch.from_numpy(future_traj_padded).type(torch.float)

        return {'past_traj': past_traj, 'future_traj': future_traj}

# ============================
# Collate Function
# ============================

def seq_collate(batch):
    """
    Collate function to handle batching of trajectories.

    Args:
        batch (list): List of samples from the dataset.

    Returns:
        dict: Batch of past and future trajectories.
    """
    past_traj = torch.stack([item['past_traj'] for item in batch])
    future_traj = torch.stack([item['future_traj'] for item in batch])
    return {'past_traj': past_traj, 'future_traj': future_traj}

# ============================
# Custom Loss Function
# ============================

class MinMSELoss(nn.Module):
    def __init__(self, reduction='mean'):
        super(MinMSELoss, self).__init__()
        self.reduction = reduction
        self.mse = nn.MSELoss(reduction='none')

    def forward(self, inputs, targets):
        """
        Args:
            inputs: Tensor of shape [batch_size, max_agents, num_preds, pred_len, 2]
            targets: Tensor of shape [batch_size, max_agents, pred_len, 2]
        Returns:
            Scalar loss
        """
        # Compute MSE loss without reduction
        mse = self.mse(inputs, targets.unsqueeze(2))  # Shape: [batch_size, max_agents, num_preds, pred_len, 2]
        mse = mse.mean(dim=-1)  # Shape: [batch_size, max_agents, num_preds, pred_len]
        mse = mse.mean(dim=-1)  # Shape: [batch_size, max_agents, num_preds]

        # Select the minimum loss across predictions
        min_mse, _ = mse.min(dim=-1)  # Shape: [batch_size, max_agents]

        if self.reduction == 'mean':
            return min_mse.mean()
        elif self.reduction == 'sum':
            return min_mse.sum()
        else:
            return min_mse

# ============================
# Configuration Loading
# ============================

# Load model configuration
config = load_config(CONFIG_PATH)

# ============================
# Model Loading Function
# ============================

def load_model(checkpoint_path, opts):
    print("[INFO] Loading model from checkpoint...")
    args = opts  # opts is already a SimpleNamespace
    model = MART(args)  # Initialize model
    try:
        checkpoint = torch.load(checkpoint_path, map_location='cpu', weights_only=True)
    except TypeError:
        # If weights_only is not supported, suppress the warning
        warnings.filterwarnings("ignore", category=FutureWarning)
        checkpoint = torch.load(checkpoint_path, map_location='cpu')
    # Load state dict
    if 'model_state_dict' in checkpoint:
        model.load_state_dict(checkpoint['model_state_dict'])
    elif 'state_dict' in checkpoint:
        model.load_state_dict(checkpoint['state_dict'])
    else:
        raise KeyError("Checkpoint does not contain 'model_state_dict' or 'state_dict'.")
    print("[INFO] Model loaded successfully!")
    return model

# ============================
# Base Model Loading
# ============================

def load_base_model(checkpoint_path, config):
    print("[INFO] Loading base model from checkpoint...")
    model = load_model(checkpoint_path, config)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    return model

base_model = load_base_model(CHECKPOINT_PATH, config)

# ============================
# Evaluation Functions
# ============================

def evaluate_model(model, test_loader, opts):
    print("[INFO] Starting evaluation...")
    ade_sum, fde_sum, total_agents = 0.0, 0.0, 0

    # Set the device to GPU or CPU
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)  # Move model to device

    model.eval()  # Set model to evaluation mode
    with torch.no_grad():
        for sample in test_loader:
            inputs = sample['past_traj'].to(device)    # Shape: [batch_size, max_agents, obs_len, 2]
            targets = sample['future_traj'].to(device)   # Shape: [batch_size, max_agents, pred_len, 2]

            batch_size, num_agents, length, _ = inputs.size()

            x_rel = torch.zeros_like(inputs).to(device)
            x_rel[:, :, 1:] = inputs[:, :, 1:] - inputs[:, :, :-1]
            x_rel[:, :, 0] = x_rel[:, :, 1]

            y_pred = model(inputs, x_rel)

            if opts.pred_rel:
                cur_pos = inputs[:, :, [-1]].unsqueeze(2)
                y_pred = torch.cumsum(y_pred, dim=3) + cur_pos

            ade = torch.min(
                torch.mean(torch.norm(y_pred - targets[:, :, None], dim=-1), dim=3),
                dim=2
            )[0].mean().item()
            fde = torch.min(
                torch.mean(torch.norm(y_pred[:, :, :, -1:] - targets[:, :, None, -1:], dim=-1), dim=3),
                dim=2
            )[0].mean().item()

            ade_sum += ade * num_agents * batch_size
            fde_sum += fde * num_agents * batch_size
            total_agents += num_agents * batch_size

    ade_avg = (ade_sum / total_agents) * opts.scale
    fde_avg = (fde_sum / total_agents) * opts.scale

    print(f"[INFO] Evaluation Results: ADE = {ade_avg:.4f}, FDE = {fde_avg:.4f}")
    return ade_avg, fde_avg

def evaluate_finetuned_model(model, test_loader, opts):
    print("[INFO] Starting evaluation of the fine-tuned model...")
    ade_sum, fde_sum, total_agents = 0.0, 0.0, 0

    model.eval()
    with torch.no_grad():
        for sample in test_loader:
            inputs = sample['past_traj'].to(device)    # Shape: [batch_size, max_agents, obs_len, 2]
            targets = sample['future_traj'].to(device)   # Shape: [batch_size, max_agents, pred_len, 2]

            batch_size, num_agents, length, _ = inputs.size()

            x_rel = torch.zeros_like(inputs).to(device)
            x_rel[:, :, 1:] = inputs[:, :, 1:] - inputs[:, :, :-1]
            x_rel[:, :, 0] = x_rel[:, :, 1]

            y_pred = model(inputs, x_rel)

            if opts.pred_rel:
                cur_pos = inputs[:, :, [-1]].unsqueeze(2)
                y_pred = torch.cumsum(y_pred, dim=3) + cur_pos

            ade = torch.min(
                torch.mean(torch.norm(y_pred - targets[:, :, None], dim=-1), dim=3),
                dim=2
            )[0].mean().item()
            fde = torch.min(
                torch.mean(torch.norm(y_pred[:, :, :, -1:] - targets[:, :, None, -1:], dim=-1), dim=3),
                dim=2
            )[0].mean().item()

            ade_sum += ade * num_agents * batch_size
            fde_sum += fde * num_agents * batch_size
            total_agents += num_agents * batch_size

    ade_avg = (ade_sum / total_agents) * opts.scale
    fde_avg = (fde_sum / total_agents) * opts.scale

    print(f"[INFO] Fine-Tuned Model Evaluation Results: ADE = {ade_avg:.4f}, FDE = {fde_avg:.4f}")
    return ade_avg, fde_avg

# ============================
# Weight Averaging Function
# ============================

def average_weights(weight_list):
    avg_weights = OrderedDict()
    for key in weight_list[0].keys():
        # Stack weights for the current key from all models and compute the mean
        avg_weights[key] = torch.mean(torch.stack([weights[key] for weights in weight_list]), dim=0)
    return avg_weights

# ============================
# Evaluation of Individual Fine-Tuned Models
# ============================

# Directory where fine-tuned checkpoints are saved
checkpoint_dir = "FineTunedCheckpoints"  # Update this if different

# Ensure the directory exists
if not os.path.isdir(checkpoint_dir):
    raise ValueError(f"Checkpoint directory '{checkpoint_dir}' does not exist. Please check the path.")

def evaluate_individual_checkpoints(checkpoint_dir, test_loader, config):
    print("\nEvaluating each fine-tuned model on the test set:")

    # List all checkpoint files in the directory
    checkpoint_files = [f for f in os.listdir(checkpoint_dir) if f.endswith('.pth')]

    if not checkpoint_files:
        raise ValueError(f"No checkpoint files found in directory '{checkpoint_dir}'.")

    for checkpoint_file in checkpoint_files:
        subset_name = checkpoint_file.replace('_checkpoint.pth', '').replace('.pth', '')
        checkpoint_path = os.path.join(checkpoint_dir, checkpoint_file)

        if not os.path.exists(checkpoint_path):
            print(f"[ERROR] Checkpoint not found for subset '{subset_name}' at '{checkpoint_path}'. Skipping...")
            continue

        # Initialize a new model instance
        model = MART(config)
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        model.to(device)

        # Load the checkpoint
        state_dict = torch.load(checkpoint_path, map_location=device)
        if isinstance(state_dict, dict) and 'model_state_dict' in state_dict:
            model.load_state_dict(state_dict['model_state_dict'])
        else:
            model.load_state_dict(state_dict)

        print(f"\n[INFO] Evaluating model fine-tuned on subset: '{subset_name}'")
        ade, fde = evaluate_model(model, test_loader, config)
        print(f"[RESULT] Subset '{subset_name}' Evaluation: ADE = {ade:.4f}, FDE = {fde:.4f}")

    print("\n[INFO] All individual fine-tuned models have been evaluated.")

# ============================
# Main Execution
# ============================

if __name__ == "__main__":
    # Prepare the test dataset and DataLoader
    test_dataset = TrajectoryDataset(
        obs_len=config.past_length,
        pred_len=config.future_length,
        mode="test",  # Correctly handled in your dataset
        scale=config.scale,
        inputs=config.inputs,
        data_dir=TEST_DATA_PATH,
        delim='\t'
    )
    test_loader = DataLoader(
        test_dataset,
        batch_size=BATCH_SIZE,
        shuffle=False,
        num_workers=4,  # Adjust based on your system
        collate_fn=seq_collate  # Ensure seq_collate is properly defined
    )

    # ============================
    # Evaluate Individual Fine-Tuned Models
    # ============================

    evaluate_individual_checkpoints(checkpoint_dir, test_loader, config)


Thu Jan 16 14:38:40 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P0              26W /  70W |   4473MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
<ipython-input-13-644746dc55ab>:343: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary


[WARNING] Averaged checkpoint 'files/eth_ucy_finetuned.pth' not found. Skipping averaged model evaluation.

Evaluating each fine-tuned model on the test set:
[INFO] PRT Agg: cat
[INFO] HRT Agg: avg
[INFO] Binary Threshold Function Type: 2

[INFO] Evaluating model fine-tuned on subset: 'zara1'
[INFO] Starting evaluation...
[INFO] Evaluation Results: ADE = 7.9592, FDE = 13.2223
[RESULT] Subset 'zara1' Evaluation: ADE = 7.9592, FDE = 13.2223
[INFO] PRT Agg: cat
[INFO] HRT Agg: avg
[INFO] Binary Threshold Function Type: 2

[INFO] Evaluating model fine-tuned on subset: 'averaged_model'
[INFO] Starting evaluation...
[INFO] Evaluation Results: ADE = 3.9677, FDE = 6.3662
[RESULT] Subset 'averaged_model' Evaluation: ADE = 3.9677, FDE = 6.3662
[INFO] PRT Agg: cat
[INFO] HRT Agg: avg
[INFO] Binary Threshold Function Type: 2

[INFO] Evaluating model fine-tuned on subset: 'zara2'
[INFO] Starting evaluation...
[INFO] Evaluation Results: ADE = 5.2156, FDE = 8.0039
[RESULT] Subset 'zara2' Evaluation: